In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from collections import Counter
import pickle

In [ ]:
def load_data(file, year):
    
    '''
    
    Loading PROMICE data for a given station and all or given year(s)
    into a DataFrame. 
    
    
    INTPUTS:
        file: Path to the desired file containing PROMICE data [string]
        year: Year to import. If 'all', all the years are imported [int,string]
    
    OUTPUTS:
        promice_data: Dataframe containing PROMICE data for the desired settings [DataFrame]
        
    '''
        
    #extract site name
    global site
    site=file.split('/')[-1].split('_')[0]+'_'+file.split('/')[-1].split('_')[1]
    
    if site[:3]=='MIT' or site[:3]=='EGP' or site[:3]=='CEN':
        site=site.split('_')[0]
    
    #load data
    promice_data=pd.read_csv(file, delim_whitespace=True)
    
    #set invalid values (-999) to nan 
    promice_data[promice_data==-999.0]=np.nan
    
    #only keep selected year(s) if needed
    global yr; yr=year
    
    if year!='all':
        promice_data=promice_data[promice_data.Year==year]
    elif isinstance(year, list):
        promice_data[promice_data.Year.isin(year)]
    
    if promice_data.empty:
        print('ERROR: Selected year not available')
        return
    
    
    return promice_data